In [1]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
import sympy
from scipy.linalg import expm
import scipy.constants as sc
from tqdm import tqdm_notebook
from scipy.fft import fft,ifft,fftfreq, dct, idct


C:\Users\david\anaconda3\lib\site-packages\qutip\__init__.py:93: UserWarning: Old version of Cython detected: needed 0.29.20, got 0.29.15.
  .format(_cy_require, _Cython.__version__)


In [12]:
H.__dict__.keys()

dict_keys(['_isherm', '_type', 'superrep', '_isunitary', 'dims', '_data'])

In [7]:

def L_S(H_S):
    si = qeye(2)
    op_list = []
    for m in range(N):
            op_list.append(si) ##creates a list of identitiy operators for each particle
    op_list = tensor(op_list)
    L = -1j*tensor(H,op_list) +1j*tensor(op_list,H.trans())
    return L 


# def L(H_S,H_LS,A_pw,gamma_p_w):
#     ###do sums
    
    
#     si = qeye(2)
#     op_list = []
#     for m in range(N):
#             op_list.append(si) ##creates a list of identitiy operators for each particle
#     op_list = tensor(op_list)
#     L = -1j*tensor(H,op_list) +1j*tensor(op_list,H.trans())
#     return L 

#'''''''''''''''''Level Statistics functions''''''''''''''''''''''
def z_k(E_array):
    z_k = [] ##setting up ratio array
    
    for i in range(np.size(E_array)):
        #_sorted_i= np.sort(E_array-E_array[i])
        E_idx_sorted = np.argsort(np.abs(E_array-E_array[i]))
        NN = E_array[E_idx_sorted[1]]
        NNN = E_array[E_idx_sorted[2]]
        
        z_k.append((NN-E_array[i])/(NNN-E_array[i]))
    z_k = np.asarray(z_k) 
    return z_k
def h_rand(h,N):
    out = np.random.uniform(-h,h,N)
    return out

def pdf_val(value,ratios,width):
    ##sorting in terms of size(only works for real)
    srtd_rts = np.sort(ratios)
    
    ##finding the indices of the values that lie at 
    ##the top and bottom of the small section centred on the value
    upper_bound = np.argmin(np.abs(srtd_rts-value-width))
    
    lower_bound = np.argmin(np.abs(srtd_rts-value+width))
    
    rho = ((upper_bound-lower_bound)/np.size(ratios))/(2*width)
    ###rho gives the number of values in the range (value-width) to (value+width)
    ##divided by the range
    return rho
def pdf(ratios,nv):
    pdf_list = []
    delta = (np.max(ratios) - np.min(ratios))/nv
    width = delta/2
    values = np.linspace(np.min(ratios),np.max(ratios),nv)
    for i in range(nv):
        pdf_val_i = pdf_val(values[i],ratios,width)
        #print(pdf_val_i)
        pdf_list.append(pdf_val_i)
    
    pdf = np.asarray(pdf_list)
    return pdf,values    






""""""""""""""""""'Functions for diagonalisation and setting up solution'""""""""""""""""""""

def H_s(N,h,J,b,t,tau):
    ####gamma is an array of 5 constants that control the dissipation
    si = qeye(2) #identity for a spin-1/2 particle
    sz = sigmaz()
    sx = sigmax()
    sigp = sigmap()
    sigm = sigmam()
    sz_list = []
    sx_list = []
    sigp_list = []
    sigm_list = []
    h_arr = h_rand(h,N)

    for n in range(N):
        op_list = []
        for m in range(N):
            op_list.append(si) ##creates a list of identitiy operators for each particle

        op_list[n] = sz
        sz_list.append(tensor(op_list))
        
        op_list[n] = sx
        sx_list.append(tensor(op_list))
        ###could get greater efficiency by just calculating the plus/minus 
        ### at the boundaries where they're needed
        if n==0:
            op_list[n] = sigp
            sigp_list.append(tensor(op_list))
        
            op_list[n] = sigm
            sigm_list.append(tensor(op_list))
        if n==N-1:
            op_list[n] = sigp
            sigp_list.append(tensor(op_list))
        
            op_list[n] = sigm
            sigm_list.append(tensor(op_list))
            
    # construct the hamiltonian 
    H = 0
    A = 0 ##bulk dephasing and amplitude damping at the boundaries
    
    # energy splitting terms
    for n in range(N):
        H += h_arr[n] * sz_list[n]
        
        ##bulk dephasing
        #A += np.sqrt(gamma[0]) * sz_list[n]
    
    ##amplitude damping
   # A += gamma[1]*sigp_list[0]+gamma[2]*sigp_list[1] + gamma[3]*sigm_list[0] + gamma[4]*sigm_list[1]
     
        ##boundary condition for ends of the chain?
    # interaction terms
    for n in range(N-1): 
        H += J * sz_list[n] * sz_list[n+1]
    
    H1 =H
    ##kicking term
    Hk = 0
    for n in range(N):
            Hk += b * sx_list[n]
    
    ##if t=tau/2, then this introduces the kicking term
    if (np.round(2*t/tau,3))==1:
        H += Hk
    
    ###Floquet operator
    U_T = (-1j*Hk).expm()*(-1j*H1).expm()
    
    
    return H,U_T,H1,Hk,A
def gamma_p_w(B,lamda):
    #out = #fourier transform of B(\tau) (autocorrelation function of bath)
    
    out = lamda**2 * B
    return out


def U_t_list(Hk,H1,Nt,t_spec): 
    t = np.linspace(0,1,Nt)
    U_t_list = []
    
    for i in range(Nt):
        U_t_spec = (-1j*Hk*t_spec).expm()*(-1j*H1/2).expm()
        out = (-1j*Hk*t[i]).expm()*(-1j*H1/2).expm()
        if i==Nt-1:
            ## if t=1, the integral is over a full delta function in total
            out = ((-1j*Hk*t[i]).expm()*(-1j*H1).expm())[:] 
        
        U_t_list.append(out)
        ### this uses the assumption that
    #integrating over half a delta function gives 1/2
    return U_t_list, U_t_spec




def Floq_modes_t0(U_T,Nt):
    floq_evecs = U_T.eigenstates()[1]
    floq_evals = U_T.eigenstates()[0]
    quasi_energies = -sc.hbar*np.angle(floq_evals)
    
    return floq_evecs,floq_evals,quasi_energies  
# Nt = 11
# U_t = U_t(Hk,H1,Nt)
# floq_modes = Floq_modes_t0(U,Nt)[0]

    
    
def Floq_mode_propagation(U_t,Nt,floq_evecs):
    ##dimension of eigenvectors
    m = np.size(floq_evecs[0]) 
    
    ##setting up for a 3D array of dimensions:
    #dimension of system hilbert space x number of eigenvectors x number of time points
    mode_array = np.zeros((Nt,m,m)) 
    
    ##creates an array of dimension (Nt,m,m)
    U_t_matrix = np.rollaxis(np.dstack(U_t),-1) 
    
    ## creates an array of shape (m,m) where the second index refers to alpha
    floq_evecs_arr = np.dstack(floq_evecs)[:,0,:]
    
    ##creates an (Nt,m,m) array where the last index gives alpha and the second index is 
    ###the dimension of the basis vectors
    mode_array = np.matmul(U_t_matrix,floq_evecs_arr)
    
    ###this makes sure t=0 one is correct, but shouldnt be needed 
    ###if we move the kicking term to the middle of the time period
    mode_array[0,:,:] = floq_evecs_arr
    
    return mode_array

def floq_f_modes(mode_array):
    
    ###performs a fourier transform along the time (0) axis,
    ### creating an array of dimensions (Nt,m,m) where the first axis
    ###denotes q, the 3rd axis denotes alpha, and the second axis gives the
    ###vector.
    f_mode_array = fft(mode_array,axis=0)
    return f_mode_array

def Apw(A,p,w,f_mode_array,quasi_energies,N):
    ##creating q,q' lists
    
    ##q_dash_list = (0,1,2,...,N-1-p)
    q_dash_list = np.linspace(0,N-1-p,N-p)
     ##q_list = (p,p+1,p+2,...,N-1)
    q_dash_list = q_list - p 
    
    ##creating e_a, e_b lists (quasienergy lists),
    ##quasienergy list may have to be turned into array for indexing etc
    
    #upper limit is quasienergies[-1]-w
    upper_limit = quasi_energies[-1]-w
    upper_limit_index = np.argmin(quasi_energies-upper_limit)
    #e_a = quasi_energies[:upper_limit_index]
    e_a_list = []
    e_a_index_list = []
    e_b_list = []
    e_b_index_list = []
    ##iterates from 0 through every quasi energy until the maximum-w (maximum-w = max poss e_a)
    for i in range(upper_limit_index): 
        poss_e_a = quasi_energies[i]
        poss_e_b = poss_e_b+w
        
        ##if statement checks to see if poss_e_b is a quasi energy up to 3 decimal places
        
        if np.min(np.round(np.abs(quasi_energies-poss_e_b),3)) ==0:
            e_a_list.append(poss_e_a)
            e_a_index_list.append(i)
            e_b_list.append(poss_e_b)
            e_b_index_list.append(np.argmin(np.round(np.abs(quasi_energies-poss_e_b),3)))
    
    ###the following is a for loop method to evaluate A_pw, its clearly
    ### very inefficient, I should work out an array method that doesnt need for loops
    
    A_pw = 0
    for i in range(np.size(e_a_list)):
        for j in range(np.size(q_list)):
            
            A_pw += 0#### matrix element of A operator 
            ##multiplied by outer product of the floquet states
    return A_pw

def A_pw_list(A,f_mode_array,quasi_energies,N):
    ##finding list of possible w frequencies
    ## DO THIS
    #w_list = 
    ##finding list of possible p frequencies
    p_list = np.linspace(0,N-1,N)
    
    for i in range(np.size(w_list)):
        for j in range(np.size(p_list)):
            A_pw_list.append(A_pw(A,p_list[j],w_list[i],f_mode_array,quasi_energies,N))
            
    return A_pw_list
    
            
    
            
        
    
    
    
    
    
    
    
    
                         
                  

In [9]:

##setting parameters
N = 10
h = 10
J = 1
b = 1
t = 1
tau = 1
H = H_s(N,h,J,b,t,tau)[0]
U = H_s(N,h,J,b,t,tau)[1]
H1 = H_s(N,h,J,b,t,tau)[2]
Hk = H_s(N,h,J,b,t,tau)[3]
E_array = H.eigenenergies()

In [13]:
H.diag()

array([ 44.04632856,  37.7507253 ,  30.55424974, ..., -20.55424974,
       -23.7507253 , -26.04632856])

In [77]:
eps = U.eigenenergies()
U_t = U_t_list(Hk,H1,10,0.23)[0]
x = U_.eigenstates()[1]

In [105]:
floq_f_modes(Floq_mode_propagation(U_t,10,x))

array([[[-0.23248165+2.89734168e-02j,  0.00341447+7.61929444e-03j,
          0.04595985+3.58812288e-02j, ..., -0.18772475+2.13799781e-01j,
         -0.00592281+2.21399686e-01j,  0.02938468-4.76127489e-02j],
        [-0.28962473+4.68963599e-01j, -0.15480581-1.71803628e-02j,
          0.02772263+9.98863279e-03j, ...,  0.03602354-1.96751904e-02j,
         -0.13890998-2.98638415e-01j, -0.0207356 -8.16831385e-02j],
        [-0.10520529-1.01572960e-01j,  0.02549324-8.20133629e-02j,
          0.16891131+3.14382586e-02j, ...,  0.05329508-1.41836808e-01j,
         -0.18339607-1.20713425e-01j,  0.06369102+1.20578648e-03j],
        ...,
        [-0.27013755-2.44309166e-01j, -0.08358594-4.27250602e-01j,
          0.04801076+1.33553828e-01j, ..., -0.12907082+1.35470324e-01j,
         -0.09652637+4.95970521e-02j, -0.15209206-1.25307607e-01j],
        [ 0.15652778+1.26695567e-01j,  0.31600785-4.73252824e-02j,
         -0.08120307-1.95370770e-01j, ...,  0.02514211+9.68097372e-02j,
          0.00405441

In [73]:
#flat_x = np.dstack(x).reshape((1024,1024))
#flat_x = np.zeros((1024,1024))
flat_x = x[:,0,:]
flat_x

array([[ 0.02487157-0.02052981j,  0.00472408-0.00389941j,
        -0.00617239-0.01665905j, ...,  0.01484506-0.0142151j ,
        -0.01433855-0.00934573j,  0.01396598-0.00350655j],
       [-0.03617967+0.07069939j, -0.00580995+0.01135333j,
        -0.00505549-0.00529169j, ...,  0.00502466-0.01190671j,
        -0.04540327-0.0077886j , -0.00333567+0.00255551j],
       [-0.02575562+0.06576673j,  0.00034652-0.00088483j,
         0.0083888 +0.0071884j , ...,  0.00555672-0.01798913j,
         0.02654096+0.00186507j, -0.02827934+0.0265288j ],
       ...,
       [-0.00449622-0.04400537j, -0.0040707 -0.03984073j,
        -0.00987211-0.00234698j, ...,  0.0075886 +0.04279823j,
         0.0332956 -0.01427093j,  0.00633599-0.01778457j],
       [-0.01103892-0.00255734j,  0.00144792+0.00033543j,
        -0.00148536+0.00235411j, ...,  0.02595546+0.0040286j ,
         0.00146964+0.0194398j ,  0.01761817+0.0140073j ],
       [ 0.00485326-0.01907773j,  0.00253656-0.00997101j,
         0.00766494+0.00506946

In [173]:
eigs = U_t_spec.eigenenergies()
eps_spec = np.exp(1j*0.23*np.angle(eps))

x = np.linspace(0,np.size(eigs)-1,np.size(eigs))

eigs_im = eigs.imag
eigs_real =eigs.real
eps_spec_im = eps_spec.imag
eps_spec_real = eps_spec.real
eps_im = eps.imag
eps_real = eps.real

im_diff = eigs_im - eps_spec_im 
real_diff = eigs_real - eps_spec_real 

plt.plot(x[:50],(np.sort(np.angle(eigs))-np.sort(np.angle(eps_spec)))[:50])
plt.show()

plt.scatter(eigs_im[:100],eigs_real[:100])
plt.xlabel('Imaginary')
plt.ylabel('Real')


plt.scatter(eps_im[:100],eps_real[:100])
plt.xlabel('Imaginary')
plt.ylabel('Real')
plt.show()


plt.scatter(eps_spec_im,eps_spec_real)
plt.xlabel('Imaginary')
plt.ylabel('Real')
plt.show()

plt.show()
#plt.scatter(im_diff,real_diff)
plt.scatter(eps_spec_im,eps_spec_real)
plt.scatter(eigs_im,eigs_real)
plt.xlabel('Imaginary')
plt.ylabel('Real')
plt.show()

# plt.plot(x,np.abs(eigs-eps_spec))
# plt.show()
ratios = z_k(E_array)

dist = pdf(ratios,100)[0]
values = pdf(ratios,100)[1]


x = np.linspace(0,1023,1024)
plt.plot(x,ratios)
plt.show()
plt.plot(x,E_array)
plt.show()
plt.plot(values,dist)
plt.ylim(0,1)
plt.show()

array([-0.99999176-0.00405892j, -0.99999005-0.00446021j,
       -0.99996685-0.00814205j, ...,  0.99996825+0.00796924j,
        0.99998992+0.00448918j,  0.99999079-0.00429095j])